In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import seaborn as sns

from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense

import xgboost as xgb

In [0]:

# Here we allow access to drives directory (to load the model)

from google.colab import drive
drive.mount('/content/drive/')

In [0]:
def parse(x):
	return datetime.strptime(x, '%Y %m %d %H')

data = pd.read_csv(r'/content/drive/My Drive/deep learning/assignment2/PRSA_data.csv', parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)

In [0]:
data = data.iloc[24:]

In [0]:
data.drop(['No'], axis=1, inplace=True)

In [0]:
data = data.fillna(0)

In [0]:
le = preprocessing.LabelEncoder()
le.fit(data['cbwd'].unique())
data['cbwd'] = le.transform(data['cbwd'])

In [0]:
min_max_norm = preprocessing.MinMaxScaler(feature_range=(0, 1))
norm_data = pd.DataFrame(min_max_norm.fit_transform(data), columns=data.columns, index=data.index)

In [0]:
def get_invers_transform(norm_data):
  return pd.DataFrame(min_max_norm.inverse_transform(norm_data), columns=norm_data.columns, index=norm_data.index)

In [0]:
'''def z_score_norm(data):
  for col in data.columns:
    mean_val = data[col].mean()
    std_val = data[col].std()
    data[col] = (data[col] - mean_val) / std_val
    
z_score_norm(data)'''

In [0]:
def eval_rmse(X_test, y_pred, y_true):
  
  # invert scaling for forecast
  temp_test = X_test.copy()
  temp_test.iloc[:,0] = y_pred
  temp_test = get_invers_transform(temp_test)
  y_pred_original_val = temp_test.iloc[:,0]
  
  # invert scaling for actual
  temp_test = X_test.copy()
  temp_test.iloc[:,0] = y_true
  temp_test = get_invers_transform(temp_test)
  y_true_original_val = temp_test.iloc[:,0]
  
  # calculate RMSE
  rmse = mean_squared_error(y_pred_original_val, y_true_original_val) ** 0.5
  print('Test RMSE: %.5f' % rmse)

# Baseline model

In [0]:
baseline_data = norm_data.copy()
baseline_data['prediction'] = norm_data['pm2.5'].rolling(window=3, min_periods=3).mean().shift(1)
baseline_data = baseline_data[3:]

In [0]:
eval_rmse(norm_data[3:], baseline_data['prediction'], baseline_data['pm2.5'])

# Transforme the dataset into a supervised learning problem

In [0]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = data.shape[1]
	df = data
	cols, names = list(), list()
  
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
      
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
  
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
    
	return agg

In [0]:
data_unsuper = series_to_supervised(norm_data)
data_unsuper.drop(data_unsuper.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
data_unsuper.index = data_unsuper.index - pd.offsets.Hour(1)

In [0]:
data_unsuper.head()

# Split to train and test setd

In [0]:
train = data_unsuper.iloc[:,:8]
target = data_unsuper.iloc[:,8]

In [0]:
train_idx = 365 * 4 * 24 # we take for training the data on the first 4 years

In [0]:
X_train = data_unsuper.iloc[:train_idx,:8]
y_train = data_unsuper.iloc[:train_idx,8]

X_test = data_unsuper.iloc[train_idx:,:8]
y_test = data_unsuper.iloc[train_idx:,8]

In [0]:
print('X_train shape', X_train.shape)
print('X_test shape', X_test.shape)

In [0]:
print('test retio', X_test.shape[0]/data_unsuper.shape[0])

# Classical ML model

In [0]:
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=4)
xgb_model.fit(X_train, y_train)

In [0]:
pred = xgb_model.predict(X_test)

In [0]:
eval_rmse(X_test, pred, y_test)

# LSTM model


In [0]:
# reshape input to be 3D [samples, timesteps, features]

X_train_val = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_val = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [0]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_val.shape[1], X_train_val.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [0]:
history = model.fit(X_train_val, y_train, epochs=50, batch_size=72, validation_data=(X_test_val, y_test), verbose=2, shuffle=False)

In [0]:
pred = model.predict(X_test_val)

In [0]:
train_pred = model.predict(X_train_val)

In [0]:
eval_rmse(X_train, train_pred, y_train)

In [0]:
eval_rmse(X_test, pred, y_test)

In [0]:
# plotting loss function

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [0]:
# pront example for bad and good predictions

# invert scaling for forecast
temp_test_1 = X_test.copy()
temp_test_1.iloc[:,0] = pred
temp_test_1 = get_invers_transform(temp_test_1)
y_pred_original_val = temp_test_1.iloc[:,0]

# invert scaling for actual
temp_test_2 = X_test.copy()
temp_test_2.iloc[:,0] = y_test
temp_test_2 = get_invers_transform(temp_test_2)
y_true_original_val = temp_test_2.iloc[:,0]

dist_df = get_invers_transform(X_test)

columns = data.columns
dist_df.columns = columns

dist_df['true_next_day_pm2.5'] = y_true_original_val
dist_df['pred_next_day_pm2.5'] = y_pred_original_val
dist_df['true_norm_val'] = y_test 
dist_df['pred_norm_val'] =  pred
dist_df['dist'] = abs(dist_df['true_norm_val']-dist_df['pred_norm_val'])
dist_df.sort_values(by=['dist'], inplace=True)


In [0]:
# good predictions

dist_df.head()

In [0]:
# bad predictions

dist_df.tail()

# PLOTS

In [0]:
plt.scatter(y_train,train_pred)
plt.title("prediction vs true value")
plt.xlabel("prediction")
plt.ylabel("true value")
plt.show()

In [0]:
sns.pairplot(data_unsuper)

In [0]:
data_unsuper.columns = np.append(norm_data.columns.values, ['pm2.5_next_day'])
data_unsuper.corr()

In [0]:
sns.distplot(norm_data['pm2.5'])